In [1]:
import pandas as pd
import numpy as np

In [2]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
basics = pd.read_csv(basics_url, sep = "\t", low_memory = False)
akas = pd.read_csv(akas_url, sep = "\t", low_memory = False)
ratings = pd.read_csv(ratings_url, sep = "\t", low_memory = False)

# Akas Processing

In [4]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33574168 entries, 0 to 33574167
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.0+ GB


In [5]:
akas.replace({"\\N":np.nan}, inplace = True)

In [6]:
akas.isna().sum()

titleId                   0
ordering                  0
title                     5
region              1866502
language            6321236
types              28244589
attributes         33325433
isOriginalTitle        2187
dtype: int64

In [7]:
akas["region"].value_counts()

JP    4007694
FR    4006579
DE    4003132
IN    3938103
ES    3929605
       ...   
CC          1
TV          1
NU          1
PW          1
NR          1
Name: region, Length: 247, dtype: int64

In [8]:
akas.query("region == 'US'", inplace = True)
akas["region"].value_counts()

US    1361717
Name: region, dtype: int64

In [9]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1361717 entries, 5 to 33573912
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1361717 non-null  object
 1   ordering         1361717 non-null  int64 
 2   title            1361717 non-null  object
 3   region           1361717 non-null  object
 4   language         3695 non-null     object
 5   types            965084 non-null   object
 6   attributes       45214 non-null    object
 7   isOriginalTitle  1360342 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.5+ MB


# Basics Processing

In [10]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9313429 entries, 0 to 9313428
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 639.5+ MB


In [11]:
basics.replace({"\\N":np.nan}, inplace = True)

In [12]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1246398
endYear           9215799
runtimeMinutes    6792331
genres             429619
dtype: int64

In [13]:
basics.dropna(subset = ["runtimeMinutes", "genres"], inplace = True)

In [14]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear           41104
endYear           2407063
runtimeMinutes          0
genres                  0
dtype: int64

In [15]:
basics["titleType"].value_counts()

tvEpisode       1107218
short            580266
movie            370350
video            176165
tvMovie           88774
tvSeries          87414
tvSpecial         16849
tvMiniSeries      16314
tvShort            9437
videoGame           308
Name: titleType, dtype: int64

In [16]:
basics.query("titleType == 'movie'", inplace = True)
basics["titleType"].value_counts()

movie    370350
Name: titleType, dtype: int64

In [17]:
basics["startYear"].value_counts()

2017    14233
2018    14175
2016    13847
2019    13838
2015    13354
        ...  
1896        1
1894        1
1899        1
1904        1
2026        1
Name: startYear, Length: 128, dtype: int64

In [18]:
basics.query("startYear >= '2000' & startYear <= '2022'", inplace = True)
basics["startYear"].value_counts()

2017    14233
2018    14175
2016    13847
2019    13838
2015    13354
2014    12998
2013    12296
2021    11845
2012    11549
2020    11295
2011    10699
2010    10135
2022     9831
2009     9290
2008     8090
2007     6897
2006     6437
2005     5774
2004     5144
2003     4539
2002     4095
2001     3821
2000     3598
Name: startYear, dtype: int64

In [19]:
basics["genres"].value_counts()

Documentary                    51553
Drama                          35232
Comedy                         13210
Comedy,Drama                    6341
Horror                          5676
                               ...  
Horror,Music,Mystery               1
Documentary,Sci-Fi,Thriller        1
Crime,Documentary,Romance          1
Adventure,History,Music            1
Crime,Fantasy,Sci-Fi               1
Name: genres, Length: 1191, dtype: int64

In [20]:
is_documentary = basics["genres"].str.contains("documentary", case = False)
basics = basics[~is_documentary]
basics["genres"].value_counts()

Drama                          35232
Comedy                         13210
Comedy,Drama                    6341
Horror                          5676
Drama,Romance                   4212
                               ...  
Animation,Music,Romance            1
Action,Animation,Game-Show         1
Horror,Reality-TV,Talk-Show        1
Family,Musical,Sport               1
Crime,Fantasy,Sci-Fi               1
Name: genres, Length: 971, dtype: int64

In [21]:
keepers = basics["tconst"].isin(akas["titleId"])
keepers

34793       True
61095       True
67641       True
77935      False
86771       True
           ...  
9313101     True
9313110     True
9313149    False
9313194     True
9313278    False
Name: tconst, Length: 144130, dtype: bool

In [22]:
basics = basics[keepers]

In [23]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83047 entries, 34793 to 9313194
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          83047 non-null  object
 1   titleType       83047 non-null  object
 2   primaryTitle    83047 non-null  object
 3   originalTitle   83047 non-null  object
 4   isAdult         83047 non-null  object
 5   startYear       83047 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  83047 non-null  object
 8   genres          83047 non-null  object
dtypes: object(9)
memory usage: 6.3+ MB


# Ratings Processing

In [24]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238963 entries, 0 to 1238962
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1238963 non-null  object 
 1   averageRating  1238963 non-null  float64
 2   numVotes       1238963 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.4+ MB


In [25]:
ratings.replace({"\\N":np.nan}, inplace = True)

In [26]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [27]:
keepers2 = ratings["tconst"].isin(akas["titleId"])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1238958    False
1238959     True
1238960    False
1238961    False
1238962    False
Name: tconst, Length: 1238963, dtype: bool

In [28]:
ratings = ratings[keepers2]

In [29]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1361717 entries, 5 to 33573912
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1361717 non-null  object
 1   ordering         1361717 non-null  int64 
 2   title            1361717 non-null  object
 3   region           1361717 non-null  object
 4   language         3695 non-null     object
 5   types            965084 non-null   object
 6   attributes       45214 non-null    object
 7   isOriginalTitle  1360342 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.5+ MB


In [30]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83047 entries, 34793 to 9313194
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          83047 non-null  object
 1   titleType       83047 non-null  object
 2   primaryTitle    83047 non-null  object
 3   originalTitle   83047 non-null  object
 4   isAdult         83047 non-null  object
 5   startYear       83047 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  83047 non-null  object
 8   genres          83047 non-null  object
dtypes: object(9)
memory usage: 6.3+ MB


In [31]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472113 entries, 0 to 1238959
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         472113 non-null  object 
 1   averageRating  472113 non-null  float64
 2   numVotes       472113 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.4+ MB


In [32]:
basics.to_csv("Data/title_basics.csv.gz", compression = 'gzip', index = False)
akas.to_csv("Data/title_akas.csv.gz", compression = 'gzip',index = False)
ratings.to_csv("Data/title_ratings.csv.gz", compression = 'gzip', index = False)

In [33]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)